In [ ]:
using MKL
using PhysicalUnits, LinearAlgebra
using MoireIVC.LLHF
using MoireIVC.LLHF_Plot
using MoireIVC.LLTDHF
using CairoMakie
CairoMakie.activate!()
Threads.nthreads()

In [ ]:
N1 = 60; N2 = 60;
num_para = LLHF_init_with_lambda(1.0; N1 = N1, N2 = N2, LL = 0);
LLHF_change_lambda!(num_para, 0.0);
symT = [LLHF.Rot3(0); LLHF.PT(0,:T)];
symP = [LLHF.Rot3(0); LLHF.PT(0,:P)];

In [ ]:
num_para.H0 .= 0.0;
ρ_C3_gapless = LLHF_solve(num_para; coherence = 0.0, 
    error_tolerance = 1E-10, max_iter_times = 100, 
    post_process_times = 100, post_procession = symT,
    complusive_mixing=false, complusive_mixing_rate=0.5, 
    stepwise_output = true, final_output = true
);

LLHF.H0_C3_T!(num_para, 0.32);
H0T = copy(num_para.H0);
ρ_C3_T = LLHF_solve(num_para; coherence = 0.0, 
    error_tolerance = 1E-10, max_iter_times = 100, 
    post_process_times = 100, post_procession = symT,
    complusive_mixing=false, complusive_mixing_rate=0.5, 
    stepwise_output = true, final_output = true
);

LLHF.H0_P!(num_para, 0.4);
H0P = copy(num_para.H0);
ρ_C3_P = LLHF_solve(num_para; coherence = 0.0, 
    error_tolerance = 1E-10, max_iter_times = 100, 
    post_process_times = 100, post_procession = symP,
    complusive_mixing=false, complusive_mixing_rate=0.5, 
    stepwise_output = true, final_output = true
);

In [ ]:
# N_arrows = 36
N_arrows = 15
dataT = LLHF_Plot.LLHF_plot_realspace_data(ρ_C3_T; para = num_para,
    N = N_arrows, xlimits = (-1.0,1.0), ylimits = (-1.0,1.0)
);
dataP = LLHF_Plot.LLHF_plot_realspace_data(ρ_C3_P; para = num_para,
    N = N_arrows, xlimits = (-1.0,1.0), ylimits = (-1.0,1.0)
);

In [ ]:
solution_C3 = Figure(size = (1300, 600));

ax_bands = Axis(solution_C3[1,1:2],
    xticks=([0,N1/3,2N1/3,N1,N1+N1÷2], ["γ", "κ", "κ'", "γ", "m"]), 
    xticksvisible = false,
    xgridcolor = :black,
    limits=((0,N1+N1÷2), nothing),
    alignmode = Outside(),
)
num_para.H0 .= 0.0;
LLHF_plot_band!(ax_bands, ρ_C3_gapless; para=num_para, unit=:meV, color=:black)
num_para.H0 .= H0P;
LLHF_plot_band!(ax_bands, ρ_C3_P; para=num_para, unit=:meV, color=:red)
num_para.H0 .= H0T;
LLHF_plot_band!(ax_bands, ρ_C3_T; para=num_para, unit=:meV, color=:blue)
Label(solution_C3[1,1:2,Top()], "  meV", 
    padding = (0.0, 0.0, 0.0, 0.0),
    halign = :left
)

ax_phase = Axis(solution_C3[1,3], aspect = DataAspect(),
    subtitle = "IVC phase winding=2",
    xticksvisible = false, xticklabelsvisible = false,
    yticksvisible = false, yticklabelsvisible = false,
)
LLHF_plot_phase!(ax_phase, ρ_C3_gapless; para=num_para)
Colorbar(solution_C3[1,4], colormap = LLHF_Plot.phase_color, 
    colorrange = (-pi,pi),
    ticks = (-pi:0.5pi:pi, ["-π","-π/2","0","π/2","π"])
)

solution_C3

In [ ]:
ax_realspace_T = Axis(solution_C3[2,1], aspect = 1.0, 
    subtitle = rich("vortex lattice(", rich("T",color=:blue), ")",
        "    S", subscript("z"), "=0"
    ),
    xticksvisible = false, yticksvisible = false,
    xgridvisible = false, ygridvisible = false,
    xticklabelsvisible = false, yticklabelsvisible = false,
)
ar_T = LLHF_plot_realspace!(ax_realspace_T, ρ_C3_T;
    para = num_para, N = N_arrows, inputdata = dataT,
    xlimits = (-1.0,1.0), ylimits = (-1.0,1.0), 
    arrowscale = 2, arrowsize = 3, arrowcolor = :n,
    text = false, linewidth = 0.5
)
Colorbar(solution_C3[2,2], ar_T, ticks=0.9:0.1:1.1)
Label(solution_C3[2,2,Top()], "δν", padding = (0.0, 0.0, 3.0, 0.0))
ax_realspace_P = Axis(solution_C3[2,3], aspect = 1.0, 
    subtitle = rich("vortex lattice(", rich("P",color=:red), ")",
        "    δν=0"
    ),
    xticksvisible = false, yticksvisible = false,
    xgridvisible = false, ygridvisible = false,
    xticklabelsvisible = false, yticklabelsvisible = false, 
)
ar_P = LLHF_plot_realspace!(ax_realspace_P, ρ_C3_P; 
    para = num_para, N = N_arrows, inputdata = dataP,
    xlimits = (-1.0,1.0), ylimits = (-1.0,1.0), 
    arrowscale = 2, arrowsize = 3, arrowcolor = :Sz,
    text = false, linewidth = 0.5
)
Colorbar(solution_C3[2,4], ar_P, ticks=-1:0.02:1)
Label(solution_C3[2,4,Top()], rich("S", subscript("z")), padding = (0.0, 0.0, 3.0, 0.0))

solution_C3

In [ ]:
ax_polarT = Axis(solution_C3[1,5], aspect = DataAspect(),
    subtitle = rich("valley polarization(", rich("T",color=:blue), ")"),
    xticksvisible = false, xticklabelsvisible = false,
    yticksvisible = false, yticklabelsvisible = false,
)
LLHF_plot_Sz!(ax_polarT, ρ_C3_T; para = num_para,
    colormap = :RdBu, colorrange = (-1,1), linecolor=:white
)
ax_polarP = Axis(solution_C3[1,6], aspect = DataAspect(),
    subtitle = rich("valley polarization(", rich("P",color=:red), ")"),
    xticksvisible = false, xticklabelsvisible = false,
    yticksvisible = false, yticklabelsvisible = false,
)
LLHF_plot_Sz!(ax_polarP, ρ_C3_P; para = num_para,
    colormap = :RdBu, colorrange = (-1,1), linecolor=:white
)
Colorbar(solution_C3[1,7], colormap = :RdBu, colorrange = (-1,1),
    ticks = -1:1
)


ax_bcT = Axis(solution_C3[2,5], aspect = DataAspect(),
    subtitle = rich("Berry curature(", rich("T",color=:blue), ")  C=0"),
    xticksvisible = false, xticklabelsvisible = false,
    yticksvisible = false, yticklabelsvisible = false,
)
num_para.H0 .= H0T;
hmt = LLHF_plot_Berrycurvature!(ax_bcT, ρ_C3_T; para = num_para, 
    colormap = :RdBu, colorrange = (-11,11), linecolor = :white
)
ax_bcP = Axis(solution_C3[2,6], aspect = DataAspect(),
    subtitle = rich("Berry curature(", rich("P",color=:red), ")  C=1"),
    xticksvisible = false, xticklabelsvisible = false,
    yticksvisible = false, yticklabelsvisible = false,
)
num_para.H0 .= H0P;
hmp = LLHF_plot_Berrycurvature!(ax_bcP, ρ_C3_P; para = num_para, 
    colormap = :RdBu, colorrange = (-11,11), linecolor = :white
)
Colorbar(solution_C3[2,7], colormap = :RdBu, colorrange = (-11,11),
    ticks = -10:10:10
)
Label(solution_C3[2,7,Top()], L"×l_B^{-2}", 
    padding = (42.0, 0.0, -8.0, 0.0),
    halign = :center
)

solution_C3

In [ ]:
for (label,i,j) in zip(["(a)", "(b)", "(c)", "(d)", "(e)"], 
    [1;2;1;1;2], [1;1;3;5;5]
)
    Label(solution_C3[i,j,TopLeft()], label, 
        fontsize = 15, font = "TeX Gyre Heros",
        padding = (0, 0, 0, 0),
    )
end

colgap!(solution_C3.layout, 0)
colgap!(solution_C3.layout, 2, -20)
colgap!(solution_C3.layout, 4, -10)
colgap!(solution_C3.layout, 5, -5)
rowgap!(solution_C3.layout, 10)

solution_C3

In [ ]:
save("C3solution.pdf", solution_C3, pt_per_unit = 1)